In [18]:
import os
import json
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
import re

# Cargar variables de entorno
load_dotenv()

# Obtener API Key de OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Archivo JSON donde guardamos las reservas
RESERVATION_FILE = "reservations.json"

# Base de datos de vuelos
air_flights = {
    "londres": {"price": "$799", "origin": "JFK (New York)", "departure_time": "10:30 AM", "date": "2025-02-15"},
    "parís": {"price": "$899", "origin": "LAX (Los Angeles)", "departure_time": "3:00 PM", "date": "2025-02-16"},
    "tokyo": {"price": "$1400", "origin": "ORD (Chicago)", "departure_time": "6:45 PM", "date": "2025-02-17"},
    "berlín": {"price": "$499", "origin": "ATL (Atlanta)", "departure_time": "8:20 AM", "date": "2025-02-18"}
}

# Funciones para manejar las reservas en JSON
def load_reservations():
    if not os.path.exists(RESERVATION_FILE):
        return {}
    with open(RESERVATION_FILE, "r", encoding="utf-8") as file:
        return json.load(file)

def save_reservation(username, destination):
    reservations = load_reservations()
    username = username.capitalize()
    
    print(f"[DEBUG] Intentando guardar reserva para {username} a {destination}")

    if username not in reservations:
        reservations[username] = []

    flight_info = air_flights.get(destination.lower())
    if not flight_info:
        print("[DEBUG] Destino no encontrado en la base de datos de vuelos")
        return "⚠️ No hay vuelos disponibles para ese destino."

    # Verificar si el usuario ya tiene una reserva para ese destino
    for ticket in reservations[username]:
        if ticket["destination"] == destination.capitalize():
            print("[DEBUG] Usuario ya tiene una reserva para este destino")
            return f"🔔 {username}, ya tienes un billete reservado para {destination.capitalize()}."

    # Crear la nueva reserva
    new_ticket = {
        "destination": destination.capitalize(),
        "origin": flight_info["origin"],
        "departure_time": flight_info["departure_time"],
        "date": flight_info["date"],
        "price": flight_info["price"]
    }
    reservations[username].append(new_ticket)

    # Sobrescribir el archivo JSON con los datos actualizados
    try:
        with open(RESERVATION_FILE, "w", encoding="utf-8") as file:
            json.dump(reservations, file, indent=4, ensure_ascii=False)
        print(f"[DEBUG] Nueva reserva guardada en JSON: {json.dumps(new_ticket, indent=4)}")
    except Exception as e:
        print(f"[ERROR] No se pudo guardar la reserva: {e}")
        return f"❌ Error al guardar la reserva: {e}"

    return (f"✅ {username}, tu reserva ha sido confirmada.\n"
            f"✈️ {new_ticket['origin']} → {new_ticket['destination']}\n"
            f"📅 {new_ticket['date']} 🕒 {new_ticket['departure_time']}\n"
            f"💰 {new_ticket['price']}")

def chat_with_openai(message, history):
    print(f"[DEBUG] Mensaje recibido: {message}")
    username, destination = extract_name_and_destination(message)
    
    if username and destination:
        response = save_reservation(username, destination)
        reservations = load_reservations()  # Recargar JSON después de guardar
    else:
        reservations = load_reservations()
    
    reservations_info = json.dumps(reservations, indent=4, ensure_ascii=False)
    
    if "consultar reservas" in message.lower() and username:
        return get_reservation(username)
    
    if destination:
        return f"✈️ Veo que quieres viajar a {destination}. ¿Cuál es tu nombre para completar la reserva?"
    elif username:
        return f"✈️ Hola {username}! Por favor dime a qué ciudad quieres viajar."

    system_prompt = (
        f"Eres un chatbot que maneja reservas de vuelos. Solo puedes realizar las siguientes acciones:\n"
        "- Mostrar los vuelos disponibles si el usuario lo solicita.\n"
        "- Reservar un vuelo cuando el usuario mencione una ciudad y su nombre.\n"
        "- Consultar las reservas de un usuario si lo solicita.\n"
        "- Saludar si el usuario te saluda.\n"
        "IMPORTANTE: No inventes información.\n"
        f"Aquí están las reservas actuales registradas: \n{reservations_info}"
    )

    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    bot_response = response.choices[0].message.content.lower()
    print(f"[DEBUG] Respuesta de OpenAI: {bot_response}")
    
    if "vuelos disponibles" in bot_response:
        return show_flights()
    
    return bot_response

# Interfaz en Gradio
with gr.Blocks() as app:
    gr.Markdown("## ✈️ Chatbot FlightAI - Reservas de Billetes")
    chat_interface = gr.ChatInterface(fn=chat_with_openai, type="messages")
    app.launch()


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


[DEBUG] Mensaje recibido: hola,  soy Carlos
[DEBUG] Mensaje recibido: Quiero viajar a París
[DEBUG] Mensaje recibido: Carlos
[DEBUG] Respuesta de OpenAI: ✈️ perfecto, carlos. se ha reservado tu vuelo a parís. si necesitas algo más, por favor házmelo saber.
[DEBUG] Mensaje recibido: Que reservas tiene Carlos?
[DEBUG] Respuesta de OpenAI: actualmente, carlos tiene las siguientes reservas:

1. tokio:
   - origen: ord (chicago)
   - hora de salida: 6:45 pm
   - fecha: 2025-02-17
   - precio: $1400

2. berlín:
   - origen: atl (atlanta)
   - hora de salida: 8:20 am
   - fecha: 2025-02-18
   - precio: $499

no veo una reserva para parís en este momento. si necesitas algo más, házmelo saber.


In [9]:
def test_load_reservations():
    reservations = load_reservations()
    print("[TEST] Cargando reservas desde JSON...")
    print(json.dumps(reservations, indent=4, ensure_ascii=False))

test_load_reservations()

[TEST] Cargando reservas desde JSON...
{
    "Carlos": [
        {
            "destination": "Tokio",
            "origin": "ORD (Chicago)",
            "departure_time": "6:45 PM",
            "date": "2025-02-17",
            "price": "$1400"
        },
        {
            "destination": "Berlín",
            "origin": "ATL (Atlanta)",
            "departure_time": "8:20 AM",
            "date": "2025-02-18",
            "price": "$499"
        }
    ],
    "Juan": [
        {
            "destination": "Berlín",
            "origin": "ATL (Atlanta)",
            "departure_time": "8:20 AM",
            "date": "2025-02-18",
            "price": "$499"
        }
    ]
}


In [13]:
def test_save_reservation():
    print("[TEST] Guardando reserva para 'Carlos' a 'París'...")
    response = save_reservation("Carlos", "París")
    print(response)

test_save_reservation()

[TEST] Guardando reserva para 'Carlos' a 'París'...
[DEBUG] Nueva reserva guardada en JSON para Carlos: {
    "destination": "Par\u00eds",
    "origin": "LAX (Los Angeles)",
    "departure_time": "3:00 PM",
    "date": "2025-02-16",
    "price": "$899"
}
✅ Carlos, tu reserva ha sido confirmada.
✈️ LAX (Los Angeles) → París
📅 2025-02-16 🕒 3:00 PM
💰 $899


In [21]:
def test_extract_name_destination():
    messages = [
        "Quiero viajar a Londres, me llamo Juan",
        "Hola, soy Ana y quiero un vuelo a París",
        "Mi nombre es Pedro y quiero ir a Tokio",
        "Quiero un vuelo a Berlín"
    ]
    
    for msg in messages:
        username, destination = extract_name_and_destination(msg)
        print(f"[TEST] Mensaje: '{msg}' → Usuario: {username}, Destino: {destination}")

test_extract_name_destination()


[TEST] Mensaje: 'Quiero viajar a Londres, me llamo Juan' → Usuario: Juan, Destino: None
[TEST] Mensaje: 'Hola, soy Ana y quiero un vuelo a París' → Usuario: Ana, Destino: París
[TEST] Mensaje: 'Mi nombre es Pedro y quiero ir a Tokio' → Usuario: Pedro, Destino: None
[TEST] Mensaje: 'Quiero un vuelo a Berlín' → Usuario: None, Destino: Berlín


In [22]:
def test_chatbot():
    messages = [
        "Quiero un vuelo a París, soy Luis",
        "Mi nombre es María y quiero viajar a Tokio",
        "Quiero viajar a Berlín"
    ]

    history = []
    for msg in messages:
        print(f"[TEST] Usuario: {msg}")
        response = chat_with_openai(msg, history)
        print(f"[TEST] Chatbot: {response}")
        history.append({"role": "user", "content": msg})
        history.append({"role": "assistant", "content": response})

test_chatbot()


[TEST] Usuario: Quiero un vuelo a París, soy Luis
[DEBUG] Mensaje recibido: Quiero un vuelo a París, soy Luis
[TEST] Chatbot: ✈️ Hola Luis! Por favor dime a qué ciudad quieres viajar.
[TEST] Usuario: Mi nombre es María y quiero viajar a Tokio
[DEBUG] Mensaje recibido: Mi nombre es María y quiero viajar a Tokio
[TEST] Chatbot: ✈️ Hola María! Por favor dime a qué ciudad quieres viajar.
[TEST] Usuario: Quiero viajar a Berlín
[DEBUG] Mensaje recibido: Quiero viajar a Berlín
[TEST] Chatbot: ✈️ Veo que quieres viajar a Berlín. ¿Cuál es tu nombre para completar la reserva?


[DEBUG] Mensaje recibido: hola
[DEBUG] Respuesta de OpenAI: ¡hola! ¿en qué puedo ayudarte hoy?
[DEBUG] Mensaje recibido: Quiero consultar unas reservas ya confirmadas
[DEBUG] Respuesta de OpenAI: lo siento, no tengo acceso a información sobre reservas ya confirmadas. pero puedo ayudarte a realizar una nueva reserva o mostrarte los vuelos disponibles. ¿te gustaría hacer alguna de estas acciones?
[DEBUG] Mostrando vuelos disponibles
